In [26]:
import openai
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter  
from langchain_openai.chat_models import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

### Parse PDF Content

In [27]:
loader = DirectoryLoader('RAG/DATA',glob="./*.pdf",loader_cls=PyPDFLoader)
documents= loader.load()
len(documents)

35

In [28]:
documents[0].page_content

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurren

In [29]:
documents[0].metadata

{'producer': 'pdfTeX-1.40.25',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2024-04-10T21:11:43+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2024-04-10T21:11:43+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'subject': '',
 'title': '',
 'trapped': '/False',
 'source': '/Users/hemchandrann/Documents/vscode/Multimodal RAG/DATA/attention.pdf',
 'total_pages': 15,
 'page': 0,
 'page_label': '1'}

In [30]:
documents[16].metadata

{'producer': 'pdfTeX-1.40.21',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2025-01-12T12:07:39-08:00',
 'author': '',
 'title': '',
 'subject': '',
 'keywords': '',
 'moddate': '2025-01-12T12:07:39-08:00',
 'trapped': '/False',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2',
 'source': '/Users/hemchandrann/Documents/vscode/Multimodal RAG/DATA/The Transformer.pdf',
 'total_pages': 20,
 'page': 1,
 'page_label': '2'}

In [31]:
len(documents[3].page_content)

2505

In [32]:
for doc in documents:
    print("<><><><><><><><><><><><><><><><><><><>")
    print("Content:", doc.page_content)
    print("<-><-><-><-><-><-><-><-><-><-><-><-><->")
    print("Metadata:", doc.metadata)
    print("<><><><><><><><><><><><><><><><><><><>")

<><><><><><><><><><><><><><><><><><><>
Content: Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensin

### Split Text into Chunks

In [33]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
texts = text_splitter.split_documents(documents)

In [34]:
len(texts)

64

In [35]:
texts[5].page_content

'Figure 1: The Transformer - model architecture.\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully\nconnected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1 Encoder and Decoder Stacks\nEncoder: The encoder is composed of a stack of N = 6 identical layers. Each layer has two\nsub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-\nwise fully connected feed-forward network. We employ a residual connection [11] around each of\nthe two sub-layers, followed by layer normalization [ 1]. That is, the output of each sub-layer is\nLayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer\nitself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding\nlayers, produce outputs of dimension dmodel = 512.\nDecoder: The decoder is also composed of a stack of N = 6ident

In [36]:
texts[5].metadata

{'producer': 'pdfTeX-1.40.25',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2024-04-10T21:11:43+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2024-04-10T21:11:43+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'subject': '',
 'title': '',
 'trapped': '/False',
 'source': '/Users/hemchandrann/Documents/vscode/Multimodal RAG/DATA/attention.pdf',
 'total_pages': 15,
 'page': 2,
 'page_label': '3'}

In [37]:
"""for i in range(len(texts)):
    print(texts[i].metadata["moddate"])"""

'for i in range(len(texts)):\n    print(texts[i].metadata["moddate"])'

In [38]:
texts[0].page_content

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurren

In [39]:
texts[27].metadata

{'producer': 'pdfTeX-1.40.21',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2025-01-12T12:07:39-08:00',
 'author': '',
 'title': '',
 'subject': '',
 'keywords': '',
 'moddate': '2025-01-12T12:07:39-08:00',
 'trapped': '/False',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2',
 'source': '/Users/hemchandrann/Documents/vscode/Multimodal RAG/DATA/The Transformer.pdf',
 'total_pages': 20,
 'page': 0,
 'page_label': '1'}

### Create the DB

In [40]:
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

### Make a retriever

In [41]:
retriever = vectordb.as_retriever()

In [42]:
docs = retriever.get_relevant_documents("what is attention?what is encoder?")
len(docs)

4

In [43]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})
retriever.search_type

'similarity'

In [44]:
retriever.search_kwargs

{'k': 5}

### Make a chain

In [45]:


# Assuming 'retriever' is already defined
retriever = vectordb.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(), 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True
)


In [46]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [47]:
# full example
query = "What is attention?.What is encoder?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


Attention is a mechanism used in neural networks to selectively focus on certain parts of the input that are relevant for the task at hand. It can be used to weight the input tokens and collect information passed from lower layers to higher layers. An encoder is a component of a neural network that takes in the input data and converts it into a form that can be used by the rest of the network. In the context of self-attention and memory networks, an encoder is responsible for encoding the input tokens and passing them to the decoder or higher layers.


Sources:
/Users/hemchandrann/Documents/vscode/Multimodal RAG/DATA/The Transformer.pdf
/Users/hemchandrann/Documents/vscode/Multimodal RAG/DATA/The Transformer.pdf
/Users/hemchandrann/Documents/vscode/Multimodal RAG/DATA/The Transformer.pdf
/Users/hemchandrann/Documents/vscode/Multimodal RAG/DATA/The Transformer.pdf


In [ ]:
# break it down
query = "list the important points of the documents "
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response["result"]

{'query': 'list the important points of the documents ',
 'result': '\n1. Contextual words can help us understand the meaning of words in a sentence or paragraph.\n2. Transformers can build contextual embeddings by integrating the meaning of these helpful contextual words.\n3. Attention is the mechanism in transformers that weighs and combines the representations of other tokens to build the representation for a specific token.\n4. Attention can be seen as a causal language model, processing the sentence from left to right.\n5. Words have rich linguistic relationships with other words, even if they are far away in the text.\n6. Attention can follow long-distance dependencies to complete phrases or concepts.\n7. A majority of American governments have passed laws since 2009 to make the registration or voting process more difficult.',
 'source_documents': [Document(metadata={'author': '', 'creationdate': '2025-01-12T12:07:39-08:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'modd

In [48]:
llm_response["result"]

'\nAttention is a mechanism used in neural networks to selectively focus on certain parts of the input that are relevant for the task at hand. It can be used to weight the input tokens and collect information passed from lower layers to higher layers. An encoder is a component of a neural network that takes in the input data and converts it into a form that can be used by the rest of the network. In the context of self-attention and memory networks, an encoder is responsible for encoding the input tokens and passing them to the decoder or higher layers.'

In [24]:
# break it down
query = "\"Many American governments have passed laws to make the registration or voting process more difficult since 2009\" explain in three points?"
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response

{'query': '"Many American governments have passed laws to make the registration or voting process more difficult since 2009" explain in three points?',
 'result': "\n1. Voter ID laws: Many American governments have passed laws requiring voters to present a valid form of identification in order to vote. This has made the registration and voting process more difficult for those who may not have easy access to identification, such as low-income individuals or those without a driver's license.\n\n2. Restrictions on early voting and absentee ballots: Some states have implemented new laws limiting the availability of early voting and absentee ballots. This makes it harder for individuals who may have difficulty getting to the polls on Election Day to participate in the voting process.\n\n3. Purging of voter rolls: In some states, there has been a trend of purging voter rolls, which removes individuals from the list of registered voters. This can happen if a voter has not participated in rece